# Crocoddyl: Contact RObot COntrol by Differential DYnamic programming Library


## I. Welcome to crocoddyl
Crocoddyl is an **optimal control library for robot control under contact sequence**. Its solver is based on an efficient Differential Dynamic Programming (DDP) algorithm. Crocoddyl computes optimal trajectories along to optimal feedback gains. It uses Pinocchio for fast computation of robot dynamics and its analytical derivatives. 

Crocoddyl is focused on multi-contact optimal control problem (MCOP) which as the form:

$$\mathbf{X}^*,\mathbf{U}^*=
\begin{Bmatrix} \mathbf{x}^*_0,\cdots,\mathbf{x}^*_N \\
				  \mathbf{u}^*_0,\cdots,\mathbf{u}^*_N
\end{Bmatrix} =
\arg\min_{\mathbf{X},\mathbf{U}} \sum_{k=1}^N \int_{t_k}^{t_k+\Delta t} l(\mathbf{x},\mathbf{u})dt$$
subject to
$$ \mathbf{\dot{x}} = \mathbf{f}(\mathbf{x},\mathbf{u}),$$
$$ \mathbf{x}\in\mathcal{X}, \mathbf{u}\in\mathcal{U}, \boldsymbol{\lambda}\in\mathcal{K}.$$
where
 - the state $\mathbf{x}=(\mathbf{q},\mathbf{v})$ lies in a manifold, e.g. Lie manifold $\mathbf{q}\in SE(3)\times \mathbb{R}^{n_j}$,
 - the system has underactuacted dynamics, i.e. $\mathbf{u}=(\mathbf{0},\boldsymbol{\tau})$,
 - $\mathcal{X}$, $\mathcal{U}$ are the state and control admissible sets, and
 - $\mathcal{K}$ represents the contact constraints.
 
 Note that $\boldsymbol{\lambda}=\mathbf{g}(\mathbf{x},\mathbf{u})$ denotes the contact force, and is dependent on the state and control.
 
Let's start by understanding the concept behind crocoddyl design.

# II. Action models

In crocoddyl, an action model combines dynamics and cost data. Each node, in our optimal control problem, is described through an action model. Every time that we want describe a problem, we need to provide ways of computing the dynamics, cost functions and their derivatives. (Note that while we have implemented NumDiff abstractions for debugging and testing, the implementation is based on **analytical derivatives**.

If we locally linearize our optimal control problem, we get:

$$\mathbf{X}^*(\mathbf{x}_0),\mathbf{U}^*(\mathbf{x}_0)
=
\arg\max_{\mathbf{X},\mathbf{U}} = cost_T(\delta\mathbf{x}_N) + \sum_{k=1}^N cost_t(\delta\mathbf{x}_k, \delta\mathbf{u}_k)$$
subject to
$$dynamics(\delta\mathbf{x}_{k+1},\delta\mathbf{x}_k,\delta\mathbf{u}_k)=\mathbf{0},$$

where
$$cost_T(\delta\mathbf{x}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{l_x}^\top \\
\mathbf{l_x} & \mathbf{l_{xx}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x}
\end{bmatrix}
$$

$$cost_t(\delta\mathbf{x},\delta\mathbf{u}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{l_x}^\top & \mathbf{l_u}^\top\\
\mathbf{l_x} & \mathbf{l_{xx}} & \mathbf{l_{ux}}^\top\\
\mathbf{l_u} & \mathbf{l_{ux}} & \mathbf{l_{uu}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}
$$

$$
dynamics(\delta\mathbf{x}_{k+1},\delta\mathbf{x}_k,\delta\mathbf{u}_k) = \delta\mathbf{x}_{k+1} - (\mathbf{f_x}\delta\mathbf{x}_k + \mathbf{f_u}\delta\mathbf{u}_k)
$$

where an action model defines a time interval of this problem:
 - $action = dynamics + cost$

## II.a Differential and Integrated Action Models
Optimal control solvers require the time-discrete model of cost and dynamics. However, there is a continuous time dynamics behind the implementation. This dynamics is implemented in the "Differential Action Model" class. Along predefined integrated action models, the user can switch between different methods for discretization.

At the moment, we have:
 - a simpletic Euler integration rule (for the dynamics), together with
 - a rectangle quadrature rule (for cost function)

An integrated action models uses a model of the form:
$$\mathbf{X}^*(\mathbf{x}_0),\mathbf{U}^*(\mathbf{x}_0)
=
\arg\max_{\mathbf{X},\mathbf{U}} = cost_T(\delta\mathbf{x}_N) + \sum_{k=1}^N \int_{t_k}^{t_k+\Delta t} cost_t(\delta\mathbf{x}_k, \delta\mathbf{u}_k) dt$$
subject to
$$dynamics(\delta\mathbf{x}_{k+1},\delta\mathbf{x}_k,\delta\mathbf{u}_k)=\mathbf{0},$$

where
$$cost_T(\delta\mathbf{x}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{l_x}^\top \\
\mathbf{l_x} & \mathbf{l_{xx}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x}
\end{bmatrix}
$$

$$cost_t(\delta\mathbf{x},\delta\mathbf{u}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{l_x}^\top & \mathbf{l_u}^\top\\
\mathbf{l_x} & \mathbf{l_{xx}} & \mathbf{l_{ux}}^\top\\
\mathbf{l_u} & \mathbf{l_{ux}} & \mathbf{l_{uu}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}
$$

$$
dynamics(\delta\mathbf{\dot{x}},\delta\mathbf{x},\delta\mathbf{u}) = \delta\mathbf{\dot{x}} - (\mathbf{f_x}\delta\mathbf{x} + \mathbf{f_u}\delta\mathbf{u})
$$


### Building a differential action model for forward dynamics
#### Loading the robot

Crocoddyl offers several robot models for benchmarking our optimal control solvers. At the time being, we have the follows: Talos arm, Talos humanoid, Talos legs and HyQ manipulators, humanoids and quadrupeds. The collection of Talos models can be downloaded in Ubuntu with the APT package *robotpkg-talos-data*.

In [42]:
from crocoddyl import loadTalosArm
import numpy as np

talos_arm = loadTalosArm()

# Defining a initial state
q0 = [0.173046, 1., -0.52366, 0., 0., 0.1, -0.005]
x0 = np.hstack([q0, np.zeros(talos_arm.model.nv)])

### calc and calcDiff
Optimal control solvers often need to compute a quadratic approximation of the action model (as previously described); this provides a search direction (computeDirection). Then it's needed to try the step along this direction (tryStep).

Typically calc and calcDiff do the precomputations that are required before computeDirection and tryStep respectively (inside the solver). These functions update the information of:
 - **calc**: update the next state and its cost value
 $$\delta\mathbf{\dot{x}}_{k+1} = \mathbf{f}(\delta\mathbf{x}_k,\mathbf{u}_k)$$
 - **calcDiff**: update the derivatives of the dynamics and cost (quadratic approximation)
 $$\mathbf{f_x}, \mathbf{f_u} \hspace{1em} (dynamics)$$
 $$\mathbf{l_x}, \mathbf{l_u}, \mathbf{l_{xx}}, \mathbf{l_{ux}}, \mathbf{l_{uu}} \hspace{1em} (cost)$$
 
 **As in Pinocchio, crocoddyl put all information inside data**, so avoiding dynamic reallocation.

In [43]:
from crocoddyl import StatePinocchio
from crocoddyl import CostModelSum
from crocoddyl import a2m
import numpy as np
import pinocchio


class DifferentialActionModelABA:
    def __init__(self,pinocchioModel):
        # The forward dynamics and its derivatives are computed with Pinocchio
        self.pinocchio = pinocchioModel
        self.nq,self.nv = self.pinocchio.nq, self.pinocchio.nv
        
        # Describes integrate, difference, Jacobian integrate and Jacobian difference
        # for any Pinocchio model
        self.State = StatePinocchio(self.pinocchio)
        
        # Keeps a stack of cost functions
        self.costs = CostModelSum(self.pinocchio)
        
        # Dimension of the state, and its tangent space, and control
        self.nx = self.State.nx
        self.ndx = self.State.ndx
        self.nout = self.nv
        self.nu = self.nv
        self.unone = np.zeros(self.nu)
    @property
    def ncost(self): return self.costs.ncost
    def createData(self): return DifferentialActionDataABA(self) # create the data needed for this model
    def calc(model,data,x,u=None):
        if u is None: u=model.unone
        nx,nu,nq,nv,nout = model.nx,model.nu,model.nq,model.nv,model.nout
        q = a2m(x[:nq]) # from np array to matrix
        v = a2m(x[-nv:]) # from np array to matrix
        tauq = a2m(u) # from np array to matrix
        
        # Computes the next state through ABA
        data.xout[:] = pinocchio.aba(model.pinocchio,data.pinocchio,q,v,tauq).flat
        
        # Updates the kinematics needed for cost computation
        pinocchio.forwardKinematics(model.pinocchio,data.pinocchio,q,v)
        pinocchio.updateFramePlacements(model.pinocchio,data.pinocchio)
        
        # Computes the cost from a set of single cost functions
        data.cost = model.costs.calc(data.costs,x,u)
        return data.xout,data.cost

    def calcDiff(model,data,x,u=None,recalc=True):
        if u is None: u=model.unone
        if recalc: xout,cost = model.calc(data,x,u)
        nx,ndx,nu,nq,nv,nout = model.nx,model.State.ndx,model.nu,model.nq,model.nv,model.nout
        q = a2m(x[:nq]) # from np array to matrix
        v = a2m(x[-nv:]) # from np array to matrix
        tauq = a2m(u) # from np array to matrix
        
        # Computes the ABA derivatives (dynamics), and keeps them inside data
        pinocchio.computeABADerivatives(model.pinocchio,data.pinocchio,q,v,tauq)
        data.Fx[:,:nv] = data.pinocchio.ddq_dq
        data.Fx[:,nv:] = data.pinocchio.ddq_dv
        data.Fu[:,:]   = pinocchio.computeMinverse(model.pinocchio,data.pinocchio,q)

        # Updates the kinematics Jacobians needed for getting the derivatives of the cost function
        pinocchio.computeJointJacobians(model.pinocchio,data.pinocchio,q)
        pinocchio.updateFramePlacements(model.pinocchio,data.pinocchio)
        
        # Computes all derivatives of cost function
        model.costs.calcDiff(data.costs,x,u,recalc=False)
        return data.xout,data.cost


class DifferentialActionDataABA:
    def __init__(self,model):
        self.pinocchio = model.pinocchio.createData()
        self.costs = model.costs.createData(self.pinocchio)
        self.cost = np.nan
        self.xout = np.zeros(model.nout)
        nx,nu,ndx,nq,nv,nout = model.nx,model.nu,model.State.ndx,model.nq,model.nv,model.nout
        self.F = np.zeros([ nout,ndx+nu ])
        self.costResiduals = self.costs.residuals
        self.Fx = self.F[:,:ndx]
        self.Fu = self.F[:,-nu:]
        self.g   = self.costs.g
        self.L   = self.costs.L
        self.Lx  = self.costs.Lx
        self.Lu  = self.costs.Lu
        self.Lxx = self.costs.Lxx
        self.Lxu = self.costs.Lxu
        self.Luu = self.costs.Luu
        self.Rx  = self.costs.Rx
        self.Ru  = self.costs.Ru

## II.b State and its integrate and difference rules
General speaking, the system's state can lie in a manifold $M$ and its tangent space $T_\mathbf{x}M$. There are few operators that needs to be defined for different rutines inside our solvers:
  - $\mathbf{x}_{k+1} = integrate(\mathbf{x}_k,\delta\mathbf{x}_k) = \mathbf{x}_k \oplus \delta\mathbf{x}_k$
  - $\delta\mathbf{x}_k = difference(\mathbf{x}_{k+1},\mathbf{x}_k) = \mathbf{x}_{k+1} \ominus \mathbf{x}_k$
 
 where $\mathbf{x}\in M$ and $\delta\mathbf{x}\in T_\mathbf{x} M$.
 
 And we also need to defined the Jacobians of these operators with respect to the first and second arguments:
  - $\frac{\partial \mathbf{x}\oplus\delta\mathbf{x}}{\partial \mathbf{x}}, \frac{\partial \mathbf{x}\oplus\delta\mathbf{x}}{\partial\delta\mathbf{x}} =Jintegrante(\mathbf{x},\delta\mathbf{x})$
  - $\frac{\partial\mathbf{x}_2\ominus\mathbf{x}_2}{\partial \mathbf{x}_1}, \frac{\partial \mathbf{x}_2\ominus\mathbf{x}_1}{\partial\mathbf{x}_1} =Jdifference(\mathbf{x}_2,\mathbf{x}_1)$

For instance, a state that lies in the Euclidean space will the typical operators:
  - $integrate(\mathbf{x},\delta\mathbf{x}) = \mathbf{x} + \delta\mathbf{x}$
  - $difference(\mathbf{x}_2,\mathbf{x}_1) = \mathbf{x}_2 - \mathbf{x}_1$
  - $Jintegrate(\cdot,\cdot) = Jdifference(\cdot,\cdot) = \mathbf{I}$
  

# TODO

# III. Solving optimal control problems with DDP

## III.a ABA dynamics for reaching a goal with Talos arm

Our optimal control solver interacts with the a formulated ShootingProblem. A shooting problem represents a stack of action models (for each knot along the trajectory).

First we need to create an action model from DifferentialActionModelABA. We use it from building terminal and running action models. In this example, we employ an simpletic Euler integration rule as follows:

In [44]:
from crocoddyl import IntegratedActionModelEuler

# Running and terminal action models
runningModel = IntegratedActionModelEuler(DifferentialActionModelABA(talos_arm.model))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelABA(talos_arm.model))

# Defining the time duration for running action models and the terminal one
dt = 1e-3
runningModel.timeStep = dt

Next we define the set of cost functions for this problem. For this particular example, we formulate three running-cost functions: 
 - goal-tracking cost, $log(^fXd_o \,^oX_f)$
   
 - state and control regularization; and $\|\mathbf{x}-\mathbf{x}_{ref}\|, \|\mathbf{u}\|$

one terminal-cost:
 - goal cost. $\|\mathbf{u}_T\|$
 
 First, let's create the common cost functions.

In [45]:
from crocoddyl import CostModelFramePlacement, CostModelState, CostModelControl
from crocoddyl import m2a
from pinocchio.utils import *
import pinocchio

# Goal tracking cost
frameName = 'gripper_left_joint' #gripper_left_fingertip_2_link'
state = StatePinocchio(talos_arm.model)
SE3ref = pinocchio.SE3(np.eye(3), np.array([ [.0],[.0],[.4] ]))
goalTrackingCost = CostModelFramePlacement(talos_arm.model,
                                       nu=talos_arm.model.nv,
                                       frame=talos_arm.model.getFrameId(frameName),
                                       ref=SE3ref)

# State and control regularization
xRegCost = CostModelState(talos_arm.model,
                          state,
                          ref=state.zero(),
                          nu=talos_arm.model.nv)
uRegCost = CostModelControl(talos_arm.model,nu=talos_arm.model.nv)

# Adds the running and terminal cost functions
runningCostModel = runningModel.differential.costs
runningCostModel.addCost( name="pos", weight = 1e-3, cost = goalTrackingCost)
runningCostModel.addCost( name="regx", weight = 1e-7, cost = xRegCost) 
runningCostModel.addCost( name="regu", weight = 1e-7, cost = uRegCost)
terminalCostModel = terminalModel.differential.costs
terminalCostModel.addCost( name="pos", weight = 1, cost = goalTrackingCost)


# Let's compute the cost and its derivatives
robot_data = talos_arm.model.createData() # Pinocchio data
data = goalTrackingCost.createData(robot_data)

# Update kinematics
q = pinocchio.randomConfiguration(talos_arm.model)
v = rand(talos_arm.model.nv)
x = m2a(np.concatenate([q,v]))
u = m2a(rand(talos_arm.model.nv))
pinocchio.forwardKinematics(talos_arm.model,robot_data,q,v)
pinocchio.computeJointJacobians(talos_arm.model,robot_data,q)
pinocchio.updateFramePlacements(talos_arm.model,robot_data)

print 'cost =', goalTrackingCost.calc(data, x, u)
print 'cost =', goalTrackingCost.calcDiff(data, x, u)
print
print 'lx =', data.Lx
print 'lu =', data.Lu
print
print 'lxx =', data.Lxx
print 'luu =', data.Luu

cost = 4.26525763979
cost = 4.26525763979

lx = [-0.62075106 -2.53106569 -0.14153276 -0.73059923 -0.81062737  2.56738988
 -0.48616739  0.          0.          0.          0.          0.          0.
  0.        ]
lu = 0

lxx = [[ 2.16948173 -0.23185495 -1.64501434  1.30568402 -1.54833054 -0.20535751
  -1.88498095  0.          0.          0.          0.          0.          0.
   0.        ]
 [-0.23185495  1.34379228  0.01814229 -0.41112851  0.25528942 -1.16141492
   0.35257016  0.          0.          0.          0.          0.          0.
   0.        ]
 [-1.64501434  0.01814229  2.19397469 -0.02310117  2.0949664   0.49341178
   1.94639534  0.          0.          0.          0.          0.          0.
   0.        ]
 [ 1.30568402 -0.41112851 -0.02310117  2.11978019 -0.07305408  0.46584694
  -0.84801518  0.          0.          0.          0.          0.          0.
   0.        ]
 [-1.54833054  0.25528942  2.0949664  -0.07305408  2.12715521  0.24630137
   2.02916086  0.          0.   

We create a trajectory with 250 knots

In [46]:
from crocoddyl import ShootingProblem

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 250
problem = ShootingProblem(x0, [ runningModel ]*T, terminalModel)

Onces we have defined our shooting problem, we create a DDP solver object and pass some callback functions for analysing  its performance.

Please note that:
- CallbackDDPLogger: store the solution information.
- CallbackDDPVerbose(level): printing message during the iterates.
- CallbackSolverDisplay(robot,rate): display the state trajectory using Gepetto viewer.

In [47]:
from crocoddyl import SolverDDP
from crocoddyl import CallbackDDPLogger, CallbackDDPVerbose, CallbackSolverDisplay

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
cameraTF = [2., 2.68, 0.54, 0.2, 0.62, 0.72, 0.22]
ddp.callback = [CallbackDDPLogger(), CallbackDDPVerbose(1), CallbackSolverDisplay(talos_arm,4,cameraTF)]

# Solving it with the DDP algorithm
ddp.solve()

# Printing the reached position
log = ddp.callback[0]
frame_idx = talos_arm.model.getFrameId(frameName)
xT = log.xs[-1]
qT = np.asmatrix(xT[:talos_arm.model.nq]).T
print
print "The reached pose by the wrist is"
print talos_arm.framePlacement(qT, frame_idx)

iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha 	   dV-exp 	  dV
   0  1.73145e+00  9.46566e-08  5.30213e-01  1.00000e-09  1.00000e-09  1.0000  2.65106e-01  -9.41894e-01
   1  1.00281e+00  4.46394e-07  2.99028e+00  1.00000e-09  1.00000e-09  1.0000  1.49514e+00  7.28641e-01
   2  3.54552e-01  2.53387e-07  1.59367e+00  1.00000e-09  1.00000e-09  1.0000  7.96836e-01  6.48257e-01
   3  3.20620e-01  5.79009e-08  3.86423e-01  1.00000e-09  1.00000e-09  1.0000  1.93212e-01  3.39321e-02
   4  2.08661e-01  5.21570e-08  3.58416e-01  1.00000e-09  1.00000e-09  1.0000  1.79208e-01  1.11958e-01
   5  1.61530e-01  3.15698e-08  1.59766e-01  1.00000e-09  1.00000e-09  0.2500  3.49489e-02  4.71311e-02
   6  1.42843e-01  1.45608e-08  7.50322e-02  1.00000e-09  1.00000e-09  0.2500  1.64133e-02  1.86871e-02
   7  1.29832e-01  8.64126e-09  4.23827e-02  1.00000e-09  1.00000e-09  1.0000  2.11914e-02  1.30107e-02
   8  1.18380e-01  6.33855e-09  4.81597e-02  1.00000e-09  1.00000e-09  0.2500  1.05349e

Let's plot the results and display final trajectory

In [48]:
from crocoddyl import plotOCSolution, plotDDPConvergence

# Plotting the solution and the DDP convergence
log = ddp.callback[0]
plotOCSolution(log.xs, log.us)
plotDDPConvergence(log.costs,log.control_regs,
                   log.state_regs,log.gm_stops,
                   log.th_stops,log.steps)

# Visualizing the solution in gepetto-viewer
CallbackSolverDisplay(talos_arm)(ddp)

AttributeError: 'module' object has no attribute 'to_rgba'

## III.b Multi-Contact dynamics for biped walking (Talos legs)
In this example, we describe the multi-contact dynamics through holonomic constraints for the support legs. From the Gauss principle, we have derived the model as:
$$
\left[\begin{matrix}
 \mathbf{M} & \mathbf{J}^{\top}_c \\
 {\mathbf{J}_{c}} & \mathbf{0} \\
\end{matrix}\right]
\left[\begin{matrix}
 \dot{\mathbf{v}} \\ -\boldsymbol{\lambda}
\end{matrix}\right]
 = 
\left[\begin{matrix}
  \boldsymbol{\tau} - \mathbf{h} \\
  -\dot{\mathbf{J}}_c \mathbf{v} \\
\end{matrix}\right]$$.


Base on a predefined walking gait, we build per each phase a specific multi-contact dynamics. Indeed we need to describe multi-phase optimal control problem. One can formulate the multi-contact optimal control problem (MCOP) as follows:


$$\mathbf{X}^*,\mathbf{U}^*=
\begin{Bmatrix} \mathbf{x}^*_0,\cdots,\mathbf{x}^*_N \\
				  \mathbf{u}^*_0,\cdots,\mathbf{u}^*_N
\end{Bmatrix} =
\arg\min_{\mathbf{X},\mathbf{U}} \sum_{p=0}^P \sum_{k=1}^{N(p)} \int_{t_k}^{t_k+\Delta t} l_p(\mathbf{x},\mathbf{u})dt$$
subject to
$$ \mathbf{\dot{x}} = \mathbf{f}_p(\mathbf{x},\mathbf{u}), \text{for } t \in [\tau_p,\tau_{p+1}]$$

$$ \mathbf{g}(\mathbf{v}^{p+1},\mathbf{v}^p) = \mathbf{0}$$

$$ \mathbf{x}\in\mathcal{X}_p, \mathbf{u}\in\mathcal{U}_p, \boldsymbol{\lambda}\in\mathcal{K}_p.$$

where $\mathbf{g}(\cdot,\cdot,\cdot)$ describes the contact dynamics, and they represents terminal constraints in each walking phase. In this example we use the following impact model:

$$\mathbf{M}(\mathbf{v}_{next}-\mathbf{v}) = \mathbf{J}_{impulse}^T$$

$$\mathbf{J}_{impulse} \mathbf{v}_{next} = \mathbf{0}$$

$$\mathbf{J}_{c} \mathbf{v}_{next} = \mathbf{J}_{c} \mathbf{v}$$



First, let's define walking shooting problem:

In [49]:
from crocoddyl import StatePinocchio
from crocoddyl import DifferentialActionModelFloatingInContact
from crocoddyl import IntegratedActionModelEuler
from crocoddyl import CostModelSum
from crocoddyl import CostModelFramePlacement, CostModelFrameVelocity
from crocoddyl import CostModelState, CostModelControl
from crocoddyl import ActuationModelFreeFloating
from crocoddyl import ContactModel6D, ContactModelMultiple
from crocoddyl import ShootingProblem


class TaskSE3:
    def __init__(self, oXf, frameId):
        self.oXf = oXf
        self.frameId = frameId


class SimpleBipedWalkingProblem:
    """ Defines a simple 3d locomotion problem
    """
    def __init__(self, robot, rightFoot, leftFoot):
        self.robot = robot
        self.state = StatePinocchio(self.robot.model)
        self.rightFoot = rightFoot
        self.leftFoot = leftFoot

    def createProblem(self, x, stepLength, stepDuration):
        # Computing the time step per each contact phase given the step duration.
        # Here we assume a constant number of knots per phase
        numKnots = 20
        timeStep = float(stepDuration)/numKnots

        # Getting the frame id for the right and left foot
        rightFootId = self.robot.model.getFrameId(self.rightFoot)
        leftFootId = self.robot.model.getFrameId(self.leftFoot)

        # Compute the current foot positions
        q0 = a2m(x[:self.robot.nq])
        rightFootPos0 = self.robot.framePlacement(q0, rightFootId).translation
        leftFootPos0 = self.robot.framePlacement(q0, leftFootId).translation

        # Defining the action models along the time instances
        n_cycles = 2
        loco3dModel = []
        import copy
        for i in range(n_cycles):
            # swing LF phase
            leftSwingModel = \
                [ self.createContactPhaseModel(
                    timeStep,
                    rightFootId,
                    TaskSE3(
                        pinocchio.SE3(np.eye(3),
                                      np.asmatrix(a2m([ [(stepLength*k)/numKnots, 0., 0.] ]) +
                                      leftFootPos0)),
                        leftFootId)
                    ) for k in range(numKnots) ]
            
            # Double support phase
            doubleSupportModel = \
                self.createContactSwitchModel(
                    rightFootId,
                    TaskSE3(
                        pinocchio.SE3(np.eye(3),
                                      np.asmatrix(a2m([ stepLength, 0., 0. ]) +
                                      leftFootPos0)),
                        leftFootId)
                    )
            
            # swing RF phase
            rightSwingModel = \
                [ self.createContactPhaseModel(
                    timeStep,
                    leftFootId,
                    TaskSE3(
                        pinocchio.SE3(np.eye(3),
                                      np.asmatrix(a2m([ 2*(stepLength*k)/numKnots, 0., 0. ]) +
                                      rightFootPos0)),
                        rightFootId)
                    ) for k in range(numKnots) ]
            
            # Final support phase
            finalSupport = \
                self.createContactSwitchModel(
                    leftFootId,
                    TaskSE3(
                        pinocchio.SE3(np.eye(3),
                                      np.asmatrix(a2m([ 2*stepLength, 0., 0. ]) +
                                      rightFootPos0)),
                        rightFootId),
                    )
            rightFootPos0 += np.asmatrix(a2m([ stepLength, 0., 0. ]))
            leftFootPos0 += np.asmatrix(a2m([ stepLength, 0., 0. ]))
            loco3dModel += leftSwingModel + [ doubleSupportModel ] + rightSwingModel + [ finalSupport ]

        problem = ShootingProblem(x, loco3dModel, finalSupport)
        return problem

    def createContactPhaseModel(self, timeStep, contactFootId, footSwingTask):
        # Creating the action model for floating-base systems. A walker system 
        # is by default a floating-base system
        actModel = ActuationModelFreeFloating(self.robot.model)

        # Creating a 6D multi-contact model, and then including the supporting
        # foot
        contactModel = ContactModelMultiple(self.robot.model)
        contactFootModel = \
            ContactModel6D(self.robot.model, contactFootId, ref=None)
        contactModel.addContact('contact', contactFootModel)

        # Creating the cost model for a contact phase
        costModel = CostModelSum(self.robot.model, actModel.nu)
        footTrack = CostModelFramePlacement(self.robot.model,
                                        footSwingTask.frameId,
                                        footSwingTask.oXf,
                                        actModel.nu)
        stateReg = CostModelState(self.robot.model,
                                  self.state,
                                  self.state.zero(),
                                  actModel.nu)
        stateReg.weights = \
            np.array([0]*6 + [0.01]*(self.robot.model.nv-6) + [10]*self.robot.model.nv)
        ctrlReg = CostModelControl(self.robot.model, actModel.nu)
        costModel.addCost("footTrack", footTrack, 100.)
        costModel.addCost("stateReg", stateReg, 0.1)
        costModel.addCost("ctrlReg", ctrlReg, 0.001)

        # Creating the action model for the KKT dynamics with simpletic Euler
        # integration scheme
        dmodel = \
            DifferentialActionModelFloatingInContact(self.robot.model,
                                                     actModel,
                                                     contactModel,
                                                     costModel)
        model = IntegratedActionModelEuler(dmodel)
        model.timeStep = timeStep
        return model

    def createContactSwitchModel(self, contactFootId, swingFootTask):
        model = self.createContactPhaseModel(0., contactFootId, swingFootTask)

        impactFootVelCost = \
            CostModelFrameVelocity(self.robot.model, swingFootTask.frameId)
        model.differential.costs.addCost('impactVel', impactFootVelCost, 10000.)
        model.differential.costs['impactVel' ].weight = 100000
        model.differential.costs['footTrack' ].weight = 100000
        model.differential.costs['stateReg'].weight = 1
        model.differential.costs['ctrlReg'].weight = 0.01
        return model

This class builds action models for each locomotion phase:
  - createContactPhaseModel: defines an action model for the swing phase
  - createContactSwitchModel: defines an action model for switch knots between phases
  
Then we build a walking by combining a set of contact phases and their contact switches.

Now let's create a walking OC problem for the Talos legs.

In [50]:
from crocoddyl import loadTalosLegs
from crocoddyl import m2a
from pinocchio.utils import *

talos_legs = loadTalosLegs()
talos_legs.model.armature[6:] = 1. #No armature

# Setting up the 3d walking problem
rightFoot = 'right_sole_link'
leftFoot = 'left_sole_link'
walk = SimpleBipedWalkingProblem(talos_legs, rightFoot, leftFoot)


# Create the initial state
q = talos_legs.q0.copy()
v = zero(talos_legs.model.nv)
x = m2a(np.concatenate([q,v]))

# Solving the 3d walking problem using DDP
stepLength = 0.2
stepDuration = 0.75
ddp = SolverDDP(walk.createProblem(x, stepLength, stepDuration))
cameraTF = [3., 3.68, 0.84, 0.2, 0.62, 0.72, 0.22]
ddp.callback = [CallbackDDPLogger(), CallbackDDPVerbose(), CallbackSolverDisplay(talos_legs,4,cameraTF)]
ddp.th_stop = 1e-9
ddp.solve(maxiter=1000,regInit=.1)

# Plotting the solution and the DDP convergence
log = ddp.callback[0]
plotOCSolution(log.xs, log.us)
plotDDPConvergence(log.costs,log.control_regs,
                   log.state_regs,log.gm_stops,
                   log.th_stops,log.steps)


iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
   0  1.44103e+07  3.79296e+06  2.44611e+05  1.00000e-02  1.00000e-02  1.0000
   1  8.69155e+06  1.15297e+10  2.87774e+07  1.00000e-03  1.00000e-03  1.0000
   2  5.29669e+06  2.35426e+09  1.73644e+07  1.00000e-03  1.00000e-03  0.2500
   3  3.41756e+06  1.06951e+09  1.05774e+07  1.00000e-03  1.00000e-03  0.2500
   4  2.12145e+06  6.06173e+08  6.82165e+06  1.00000e-03  1.00000e-03  0.2500
   5  6.26204e+05  4.08479e+08  4.23209e+06  1.00000e-04  1.00000e-04  1.0000
   6  3.42399e+05  3.59390e+08  1.24658e+06  1.00000e-04  1.00000e-04  0.2500
   7  1.06493e+05  2.21961e+08  6.79557e+05  1.00000e-05  1.00000e-05  1.0000
   8  7.36529e+04  8.87612e+07  2.09061e+05  1.00000e-05  1.00000e-05  0.2500
   9  5.42833e+04  5.91903e+07  1.44459e+05  1.00000e-05  1.00000e-05  0.2500
iter 	 cost 	      theta 	    gamma 	  muV 	      muLM 	 alpha
  10  4.49412e+04  4.03724e+07  1.06991e+05  1.00000e-05  1.00000e-05  0.2500
  11  3.02996e+0

AttributeError: 'module' object has no attribute 'to_rgba'

Let's display again the optimal trajectory computed with our DDP solver:

In [51]:
CallbackSolverDisplay(talos_legs)(ddp)

## Reference

The material presented in this Notebook was previously presented at the ICHR at 2018. For more information, please read the following paper:

R. Budhiraja, J. Carpentier, C. Mastalli and N. Mansard. *Differential Dynamic Programming for Multi-Phase Rigid Contact Dynamics*

In [52]:
from IPython.display import HTML

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/X82tFTR4Mcc?start=11" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')



![title](https://cmastalli.github.io/assets/img/publications/astronaut360_v2.png)